In [1]:
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
#import chardet
import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern


import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString

import os

In [2]:
dpt_shp = gpd.read_file(r'C:\Users\HP\OneDrive - Universidad del Pacífico\textos\Escritorio\2025-1\Git\PC\PC3\shape_file\DISTRITOS.shp')

In [13]:
final_df = pd.read_excel(r'C:\Users\HP\OneDrive - Universidad del Pacífico\textos\Escritorio\2025-1\Git\PC\PC3\archivo_final.xlsx')


In [ ]:
# No Correr más de una vez
# Crear geometría de puntos a partir de las coordenadas de latitud y longitud
geometry = [Point(lon, lat) for lon, lat in zip(final_df['Longitud'], final_df['Latitud'])]

# Crear un GeoDataFrame de las escuelas
dt_colegio = gpd.GeoDataFrame(final_df, geometry=geometry)

# si es none usar set_crs, si quiere cambiar usar to_crs
# Cambiar el sistema de coordenadas a EPSG:4326 (WGS 84)
dt_colegio['geometry'] = dt_colegio['geometry'].set_crs(epsg=4326)
print(dt_colegio['geometry'].crs)

#Separar la data
dt_cole = dt_colegio[(dt_colegio['Departamento'] == 'AYACUCHO') | (dt_colegio['Departamento'] == 'HUANCAVELICA')]
dt_cole_prim = dt_cole[dt_cole['Nivel / Modalidad'].str.contains('Primaria', case=False)]
dt_cole_sec = dt_cole[dt_cole['Nivel / Modalidad'].str.contains('Secundaria', case=False)]

EPSG:4326


In [ ]:
# 4. Reprojectar a un CRS en metros (UTM zona 18S para Perú)
primarias_m = dt_cole_prim.to_crs(epsg=32718)
secundarias_m = dt_cole_sec.to_crs(epsg=32718)

# 5. Crear buffers de 5 km alrededor de las primarias
primarias_m['buffer5km'] = primarias_m.geometry.buffer(5000)  # 5000 metros = 5 km

# 6. Realizar unión espacial para encontrar secundarias dentro de los buffers
buffers = gpd.GeoDataFrame(geometry=primarias_m['buffer5km'], crs=primarias_m.crs)
secundarias_en_buffers = gpd.sjoin(secundarias_m, buffers, predicate='within')

# 7. Contar cuántas secundarias caen en cada buffer
conteo_secundarias = secundarias_en_buffers.groupby('index_right').size()
primarias_m['secundcerc'] = primarias_m.index.map(conteo_secundarias).fillna(0)



c:\Users\HP\OneDrive - Universidad del Pacífico\textos\Escritorio\2025-1\Git\streamlit\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [97]:
max_coles =  primarias_m['secundcerc'].max()
min_coles =  primarias_m['secundcerc'].min()
list_max = primarias_m[primarias_m['secundcerc']== max_coles]
list_min = primarias_m[primarias_m['secundcerc']== min_coles]
list_min = list_min.to_crs(epsg=4326)
list_max = list_max.to_crs(epsg=4326)


In [102]:
list_max

,Código Modular,Anexo,Nombre de SS.EE.,Ubigeo,Departamento,Provincia,Distrito,Código DRE/UGEL,DRE / UGEL,Centro Poblado,...,Dirección,Nivel / Modalidad,Gestion / Dependencia,Latitud,Longitud,Altitud,Fuente de coordenadas,geometry,buffer5km,secundcerc
85737,422063,0,38001 GUSTAVO CASTRO PANTOJA,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON GARCILAZO DE LA VEGA 111,Primaria,Pública - Sector Educación,-13.160380,-74.227700,2760,UBICACION_WEB (LOCAL),POINT (-74.2277 -13.16038),"POLYGON ((588697.447 8544999.912, 588673.371 8...",81.0
85747,422170,0,39001 MARISCAL SUCRE,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON DOS DE MAYO 179,Primaria,Pública - Sector Educación,-13.161930,-74.225360,2741,UBICACION_WEB (LOCAL),POINT (-74.22536 -13.16193),"POLYGON ((588950.529 8544827.708, 588926.453 8...",81.0
85748,422188,0,39002 MARIA PARADO DE BELLIDO,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON CALLAO 289,Primaria,Pública - Sector Educación,-13.159270,-74.228380,2770,UBICACION_WEB (LOCAL),POINT (-74.22838 -13.15927),"POLYGON ((588624.125 8545122.901, 588600.048 8...",81.0
85750,422246,0,39008 YURAQ YURAQ,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,AVENIDA LOS ANDES 102,Primaria,Pública - Sector Educación,-13.161130,-74.231570,2805,UBICACION_WEB (LOCAL),POINT (-74.23157 -13.16113),"POLYGON ((588277.762 8544918.249, 588253.686 8...",81.0
85765,1697374,0,A UNO,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON LIMA 357-D,Primaria,Privada - Particular,-13.159930,-74.230560,2790,UBICACION_WEB (LOCAL),POINT (-74.23056 -13.15993),"POLYGON ((588387.632 8545050.63, 588363.555 85...",81.0
85777,1609924,0,BERTOLT BRECHT,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON CALLAO 257,Primaria,Privada - Particular,-13.159370,-74.228110,2767,UBICACION_WEB (LOCAL),POINT (-74.22811 -13.15937),"POLYGON ((588653.354 8545111.751, 588629.277 8...",81.0
85780,1781640,0,CARITAS FELICES,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON LIBERTAD 680,Primaria,Privada - Particular,-13.158550,-74.228380,2768,UBICACION_WEB (LOCAL),POINT (-74.22838 -13.15855),"POLYGON ((588624.369 8545202.53, 588600.293 85...",81.0
85805,1139708,0,DISCOVERY,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON SAN MARTIN 342,Primaria,Privada - Particular,-13.161850,-74.227500,2754,UBICACION_WEB_MED (LOCAL),POINT (-74.2275 -13.16185),"POLYGON ((588718.624 8544837.269, 588694.547 8...",81.0
85808,1346048,0,EL BUEN PASTOR,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON GRAU 824,Primaria,Privada - Particular,-13.164720,-74.228720,2752,UBICACION_WEB (LOCAL),POINT (-74.22872 -13.16472),"POLYGON ((588585.428 8544520.262, 588561.351 8...",81.0
85811,614701,0,EL NAZARENO,50101,AYACUCHO,HUAMANGA,AYACUCHO,50001,HUAMANGA,AYACUCHO,...,JIRON GARCILAZO DE LA VEGA 277,Primaria,Privada - Particular,-13.158519,-74.227247,2763,MED_GPS (LOCAL),POINT (-74.22725 -13.15852),"POLYGON ((588747.2 8545205.6, 588723.124 85447...",81.0


In [116]:
import folium as fm
meanlat = dt_cole_prim['geometry'].y.mean()
meanlon = dt_cole_prim['geometry'].x.mean()

# 1. Crear el mapa centrado en Perú (ajusta el zoom y centro si quieres)
a = fm.Map(location=[meanlat, meanlon], zoom_start=8)

# 2. Añadir los centroides como marcadores
for _, row in list_min.iterrows():
    if row['geometry'] is not None:
        lat = row['geometry'].y
        lon = row['geometry'].x
        nombre = f"Nombre: {row['Nombre de SS.EE.']} y tiene 0 secundarias"
        
       
        fm.Circle(
              radius = 5000,
              fill=True,
              color="#db213a",
              fill_color="#db213a",
              location=[lat, lon],
              popup=nombre,
              fill_opacity=0.4
        ).add_to(a)


for _, row in list_max.iterrows():
    if row['geometry'] is not None:
        lat = row['geometry'].y
        lon = row['geometry'].x
        nombre = f"MAX: {int(row['secundcerc'])} secundarias y el nombre es {row['Nombre de SS.EE.']}"
        #nombre = 'Maximo: ' + row['secundcerc'].astype(str) + ' de Secundarias'
        
       
        fm.Circle(
              radius = 5000,
              fill=True,
              color="#25c22f",
              fill_color="#25c22f",
              location=[lat, lon],
              popup=nombre,
              fill_opacity=0.4
        ).add_to(a)

# Marcador con flecha personalizada (HTML/CSS)

lat_max = list_max['geometry'].y.values[0]
lon_max = list_max['geometry'].x.values[0]
fm.Marker(
    location=[lat_max, lon_max],
    icon=fm.Icon(color="green", icon="location-arrow", prefix="fa"),
    nombre = f"Nombre: {list_max['Nombre de SS.EE.'].values[0]} y tiene {int(list_max['secundcerc'].values[0])} secundarias",
    popup=nombre
).add_to(a)

lat_min = list_min['geometry'].y.values[0]
lon_min = list_min['geometry'].x.values[0]
fm.Marker(
    location=[lat_min, lon_min],
    icon=fm.Icon(color="red", icon="location-arrow", prefix="fa"),
    nombre = f"Nombre: {list_min['Nombre de SS.EE.'].values[0]} y tiene 0 secundarias",
    popup=nombre
).add_to(a)



# 3. Mostrar el mapa
a
